In [ ]:
# Cell 1: Import Libraries and Setup
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Print TensorFlow version and GPU availability
print(f"✅ TensorFlow version: {tf.__version__}")
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"🚀 GPU is available: {gpus[0].name}")
else:
    print("⚠️ No GPU found. Training will use CPU.")


In [2]:
# Cell 2: GPU Configuration and Memory Growth
# Configure GPU memory growth to avoid OOM errors
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Num GPUs Available: {len(gpus)}")
        print("GPU memory growth enabled")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU available, using CPU")

Num GPUs Available: 1
GPU memory growth enabled


In [3]:
# Cell 3: Configuration Parameters
IMAGE_H = 256
IMAGE_W = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 50
LEARNING_RATE = 0.0001

In [4]:
# Cell 4: Data Augmentation for Training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [5]:
# Cell 5: Validation Data Generator
validation_datagen = ImageDataGenerator(
    rescale=1./255
)


In [6]:
# Cell 6: Test Data Generator
test_datagen = ImageDataGenerator(
    rescale=1./255
)

In [7]:
# Cell 7: Create Data Generators
train_generator = train_datagen.flow_from_directory(
    'deepfake_final_exam/train',
    target_size=(IMAGE_H, IMAGE_W),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'deepfake_final_exam/train'